In [11]:
import pandas as pd
from addfips import AddFIPS

In [ ]:


# Load CSV
df = pd.read_csv("../data/raw/County Population Totals 2020-2024.csv")

# Remove leading dot from 'Geographic Area'
df['Geographic Area'] = df['Geographic Area'].str.lstrip('.')

# Split into County and State
df[['County', 'State']] = df['Geographic Area'].str.extract(r'^(.*?),\s*(.*)$')

# Clean 'County' to remove " County" if present (optional depending on input)
df['County'] = df['County'].str.replace(" County", "", regex=False)

# Clean '2023' population column: remove commas and convert to int
df = df.dropna(subset=['2023'])  # Drop rows with missing population
df['2023'] = df['2023'].str.replace(',', '').astype(int)

# Initialize AddFIPS
af = AddFIPS()

# Drop rows where State or County is missing (to avoid AttributeError)
df = df.dropna(subset=['State', 'County'])

# Get FIPS codes
df['FIPS'] = df.apply(lambda row: af.get_county_fips(state=row['State'], county=row['County']), axis=1)

# Drop rows where FIPS couldn't be determined
df = df.dropna(subset=['FIPS'])

# Final output: just FIPS and 2023 population
cleaned = df[['FIPS', '2023']]

cleaned.columns = ['FIPS', 'Population']

cleaned.head()


In [19]:
cleaned.to_csv("../data/cleaned/population_cleaned.csv", index=False)
